# Integrantes
- Daniel Diab
- Laura Martinez

# 1. Preparación Datos
## 1.1. Importar Librerias

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

## 1.2. Cargar Datos

In [23]:
df = pd.read_csv('./data.csv', sep=',', header=0)
datos = df.copy()

## 1.3. Eliminar Duplicados

In [24]:
import pandas as pd

def corregir_fechas(df, col="Date of Service"):
    df = df.copy()

    s = df[col].astype("string").str.strip()
    s = s.replace({"": pd.NA, "nan": pd.NA, "NaN": pd.NA, "null": pd.NA, "None": pd.NA})

    # pandas moderno: resuelve formatos mixtos de forma fiable
    try:
        dt = pd.to_datetime(s, errors="coerce", format="mixed", dayfirst=True)
    except TypeError:
        dt = pd.to_datetime(s, errors="coerce", dayfirst=True)
        m = dt.isna() & s.notna()
        dt.loc[m] = pd.to_datetime(s[m], errors="coerce", dayfirst=False)

    df[col] = dt.dt.normalize() #solo fecha, no hora
    return df

datos = corregir_fechas(datos, "Date of Service")
print("Fechas inválidas:", datos["Date of Service"].isna().sum())

Fechas inválidas: 0


In [25]:
print("Duplicados exactos:", datos.duplicated().sum())

Duplicados exactos: 151


In [26]:
datos = datos.drop_duplicates()
datos.shape

(1488, 24)

In [27]:
cols_llave = ["Patient ID", "Date of Service"]

conflict_keys = datos.loc[
    datos.duplicated(subset=cols_llave, keep=False), cols_llave].drop_duplicates()

print("Llaves conflictivas:", len(conflict_keys))

datos = datos.merge(conflict_keys, on=cols_llave, how="left", indicator=True)
datos = datos[datos["_merge"] == "left_only"].drop(columns="_merge")

print("Filas restantes:", len(datos))

Llaves conflictivas: 112
Filas restantes: 1264


## 1.4. Separar Dataset

In [28]:
from sklearn.model_selection import train_test_split

X = datos.drop(columns=["CVD Risk Score", "CVD Risk Level"], errors="ignore")
y = datos["CVD Risk Score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## 1.5. Pipeline

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer

In [31]:
cols_to_drop = ["Patient ID", "Date of Service", "CVD Risk Level", "Blood Pressure (mmgHg)"]

def drop_columns(df):
    return df.drop(columns=cols_to_drop, errors="ignore")

dropper = FunctionTransformer(drop_columns)

In [32]:
def _to_num(s):
    return pd.to_numeric(s, errors="coerce")

def completar_bp_desde_texto(df, col_bp_texto="Blood Pressure (mmHg)", col_sis="Systolic BP", col_dia="Diastolic BP"):
    df = df.copy()
    if col_bp_texto not in df.columns:
        return df

    s = df[col_bp_texto].astype("string").str.strip()
    extra = s.str.extract(r"^\s*(\d+)\s*/\s*(\d+)\s*$")
    sys = _to_num(extra[0])
    dia = _to_num(extra[1])

    if col_sis in df.columns:
        m = df[col_sis].isna() & sys.notna()
        df.loc[m, col_sis] = sys.loc[m]

    if col_dia in df.columns:
        m = df[col_dia].isna() & dia.notna()
        df.loc[m, col_dia] = dia.loc[m]

    return df

tr_completar_bp = FunctionTransformer(completar_bp_desde_texto)


def corregir_swap_bp(df, col_sis="Systolic BP", col_dia="Diastolic BP"):
    df = df.copy()
    if col_sis not in df.columns or col_dia not in df.columns:
        return df

    sis = _to_num(df[col_sis])
    dia = _to_num(df[col_dia])

    m = sis.notna() & dia.notna() & (dia > sis)
    if m.any():
        df.loc[m, col_sis], df.loc[m, col_dia] = dia.loc[m].values, sis.loc[m].values

    return df

tr_swap_bp = FunctionTransformer(corregir_swap_bp)


def completar_altura_m_cm(df, col_h_m="Height (m)", col_h_cm="Height (cm)"):
    df = df.copy()
    if col_h_m not in df.columns or col_h_cm not in df.columns:
        return df

    hm = _to_num(df[col_h_m])
    hcm = _to_num(df[col_h_cm])

    # si falta cm y hay m
    m1 = hm.notna() & hcm.isna()
    df.loc[m1, col_h_cm] = hm.loc[m1] * 100

    # si falta m y hay cm
    m2 = hm.isna() & hcm.notna()
    df.loc[m2, col_h_m] = hcm.loc[m2] / 100

    return df

tr_altura_completar = FunctionTransformer(completar_altura_m_cm)


def resolver_inconsistencia_altura(df, col_h_m="Height (m)", col_h_cm="Height (cm)", tol_cm=1.0):
    df = df.copy()
    if col_h_m not in df.columns or col_h_cm not in df.columns:
        return df

    hm = _to_num(df[col_h_m])
    hcm = _to_num(df[col_h_cm])

    m = hm.notna() & hcm.notna() & (abs(hcm - hm * 100) > tol_cm)
    if m.any():
        df.loc[m, col_h_cm] = hm.loc[m] * 100

    return df

tr_altura_resolver = FunctionTransformer(resolver_inconsistencia_altura)


def recalcular_bmi(df, col_peso="Weight", col_h_m="Height (m)", col_bmi="BMI"):
    df = df.copy()
    if col_peso not in df.columns or col_h_m not in df.columns or col_bmi not in df.columns:
        return df

    w = _to_num(df[col_peso])
    hm = _to_num(df[col_h_m])

    m = w.notna() & hm.notna() & (hm > 0)
    df.loc[m, col_bmi] = w.loc[m] / (hm.loc[m] ** 2)

    return df

tr_bmi = FunctionTransformer(recalcular_bmi)


def recalcular_waist_height_ratio(df, col_cintura="Abdominal Circumference (cm)", col_h_cm="Height (cm)", col_ratio="Waist-to-Height Ratio"):
    df = df.copy()
    if col_cintura not in df.columns or col_h_cm not in df.columns or col_ratio not in df.columns:
        return df

    cintura = _to_num(df[col_cintura])
    hcm = _to_num(df[col_h_cm])

    m = cintura.notna() & hcm.notna() & (hcm > 0)
    df.loc[m, col_ratio] = cintura.loc[m] / hcm.loc[m]

    return df

tr_ratio = FunctionTransformer(recalcular_waist_height_ratio)

def clip_iqr(df, factor=1.5):
    df = df.copy()

    numeric_cols = df.select_dtypes(include=[np.number]).columns

    for col in numeric_cols:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1

        lower = q1 - factor * iqr
        upper = q3 + factor * iqr

        df[col] = df[col].clip(lower, upper)

    return df

clipper_transformer = FunctionTransformer(clip_iqr)

In [33]:
numeric_features = [
    "Age",
    "Weight (kg)",
    "Height (m)",
    "Height (cm)",
    "BMI",
    "Abdominal Circumference (cm)",
    "Waist-to-Height Ratio",
    "Systolic BP",
    "Diastolic BP",
    "Total Cholesterol (mg/dL)",
    "HDL (mg/dL)",
    "Estimated LDL (mg/dL)",
    "Fasting Blood Sugar (mg/dL)",
]

#Variables categóricas
binary_features = [
    "Sex",
    "Smoking Status",
    "Diabetes Status",
    "Family History of CVD",
]

ordinal_features = [
    "Physical Activity Level",
]

nominal_features = [
    "Blood Pressure Category",
]

In [34]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("clipper", clipper_transformer),
    ("scaler", StandardScaler()),
])

binary_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(drop="if_binary", handle_unknown="ignore")),
])

ordinal_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ordinal", OrdinalEncoder(categories=[["Low", "Moderate", "High"]])),
])

nominal_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

In [35]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("bin", binary_transformer, binary_features),
        ("ord", ordinal_transformer, ordinal_features),
        ("nom", nominal_transformer, nominal_features),
    ],
    remainder="drop"
)

In [36]:
from sklearn.pipeline import Pipeline

pipeline_preparacion = Pipeline(steps=[
    ("bp_fill", tr_completar_bp),
    ("dropper", dropper),

    # determinísticas (en el orden correcto)
    ("bp_swap", tr_swap_bp),
    ("h_fill", tr_altura_completar),
    ("h_fix", tr_altura_resolver),
    ("bmi", tr_bmi),
    ("ratio", tr_ratio),

    # no determinísticas (aprenden en fit): imputación + clipping + escalado + encoding
    ("preprocesamiento", preprocessor),
])

pipeline_preparacion

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('bp_fill', ...), ('dropper', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"func func: callable, default=NoneThe callable to use for the transformation. This will be passedthe same arguments as transform, with args and kwargs forwarded.If func is None, then func will be the identity function.",<function com...00181D3D3CE00>
,"inverse_func inverse_func: callable, default=NoneThe callable to use for the inverse transformation. This will bepassed the same arguments as inverse transform, with args andkwargs forwarded. If inverse_func is None, then inverse_funcwill be the identity function.",None
,"validate validate: bool, default=FalseIndicate that the input X array should be checked before calling``func``. The possibilities are:- If False, there is no input validation.- If True, then X will be converted to a 2-dimensional NumPy array or sparse matrix. If the conversion is not possible an exception is raised... versionchanged:: 0.22 The default of ``validate`` changed from True to False.",False
,"accept_sparse accept_sparse: bool, default=FalseIndicate that func accepts a sparse matrix as input. If validate isFalse, this has no effect. Otherwise, if accept_sparse is false,sparse matrix inputs will cause an exception to be raised.",False
,"check_inverse check_inverse: bool, default=TrueWhether to check that or ``func`` followed by ``inverse_func`` leads tothe original inputs. It can be used for a sanity check, raising awarning when the condition is not fulfilled... versionadded:: 0.20",True
,"feature_names_out feature_names_out: callable, 'one-to-one' or None, default=NoneDetermines the list of feature names that will be returned by the`get_feature_names_out` method. If it is 'one-to-one', then the outputfeature names will be equal to the input feature names. If it is acallable, then it must take two positional arguments: this`FunctionTransformer` (`self`) and an array-like of input feature names(`input_features`). It must return an array-like of output featurenames. The `get_feature_names_out` method is only defined if`feature_names_out` is not None.See ``get_feature_names_out`` for more details... versionadded:: 1.1",None
,"kw_args kw_args: dict, default=NoneDictionary of additional keyword arguments to pas

In [39]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(948, 22)
(948,)
(316, 22)
(316,)
